In [ ]:
import csv
import json

## Open and read the geojson file and convert it to a dictionary

with open('../kml/merged_test_1.geojson') as f:
    data = json.load(f)

fields = []
for feature in data['features']:
    ff = feature['properties']['Name']
    home_plate = feature['geometry']['coordinates'][0][0]
    polygon = feature['geometry']['coordinates'][0]
    field_name = feature['properties']['layer']
    fields.append({'field_name': field_name, 'fair_foul': ff, 'home_plate': home_plate, 'polygon': polygon})

with open('baseball_fields.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Field', 'Fair/Foul', 'Home Plate', 'Polygon'])
    for field in fields:
        writer.writerow([field['field_name'], field['fair_foul'], field['home_plate'], field['polygon']])



In [ ]:
import csv
import json

with open('../kml/merged_test_1.geojson') as f:
    data = json.load(f)

fields = []
for feature in data['features']:
    ff = feature['properties']['Name']
    home_plate = feature['geometry']['coordinates'][0][0]
    polygon = feature['geometry']['coordinates'][0]
    field_name = feature['properties']['layer']
    fields.append({'field_name': field_name, 'fair_foul': ff, 'home_plate': home_plate, 'polygon': polygon})

## Output JSON
with open('output.json', 'w') as outfile:
    json.dump(fields, outfile)



## Output CSV
# with open('baseball_fields.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(['Name', 'Home Plate', 'Polygon', 'Layer'])
#     for field in fields:
#         writer.writerow([field['name'], field['home_plate'], field['polygon'], field['layer']])



## Working Above - Data reshaped

### Block below is the one I was working on last

In [2]:
###### WORKING HERE ######
import json
from shapely.geometry import Point
from shapely.geometry import LineString
import geopy.distance


# Load data from JSON file
with open('output.json', 'r') as file:
    data = json.load(file)

def remove_altitude(data):
    new_data = []
    for item in data:
        new_polygon = []
        for coordinates in item['polygon']:
            new_coordinates = coordinates[:2]  # Remove altitude value
            new_polygon.append(new_coordinates)
        item['polygon'] = new_polygon
        item['home_plate'] = item['home_plate'][:2]  # Remove altitude value from home_plate
        new_data.append(item)
    return new_data

updated_data = remove_altitude(data)

def add_distances_to_data(data):
    def get_distance(point1, point2):
        return geopy.distance.distance(point1, point2).km

    def process_item(item):
        home_lat = item['home_plate'][1]
        home_lon = item['home_plate'][0]
        home_point = Point(home_lon if not isinstance(home_lon, list) else home_lon[0], 
                           home_lat if not isinstance(home_lat, list) else home_lat[0])
        distances = []
        for polygon in item['polygon']:
            try:
                if len(polygon) > 0:
                    polygon_lat, polygon_lon = polygon[0][1], polygon[0][0]
                    polygon_point = Point(polygon_lon if not isinstance(polygon_lon, list) else polygon_lon[0], 
                                          polygon_lat if not isinstance(polygon_lat, list) else polygon_lat[0])
                    distance = get_distance(home_point, polygon_point)
                    distances.append(distance)
            except TypeError:
                print(f'Error in processing polygon: {polygon}')
        return {'home_plate': item['home_plate'], 'polygon': item['polygon'], 'distances': distances}
        
    return [process_item(item) for item in data]

data_with_distance = add_distances_to_data(updated_data)

print(data_with_distance)




Error in processing polygon: [-84.3395166, 41.9898236]
Error in processing polygon: [-84.3406566, 41.9898485]
Error in processing polygon: [-84.3406579, 41.9897867]
Error in processing polygon: [-84.3406526, 41.9897179]
Error in processing polygon: [-84.3399766, 41.989706]
Error in processing polygon: [-84.3395529, 41.989697]
Error in processing polygon: [-84.3394831, 41.989708]
Error in processing polygon: [-84.3393651, 41.9897817]
Error in processing polygon: [-84.3393423, 41.9898395]
Error in processing polygon: [-84.3393222, 41.9906699]
Error in processing polygon: [-84.3394898, 41.9906599]
Error in processing polygon: [-84.3395166, 41.9898236]
Error in processing polygon: [-84.3395166, 41.9898236]
Error in processing polygon: [-84.3394898, 41.9906599]
Error in processing polygon: [-84.3395904, 41.9906559]
Error in processing polygon: [-84.3397232, 41.9906469]
Error in processing polygon: [-84.3398278, 41.99064]
Error in processing polygon: [-84.3398828, 41.990639]
Error in process

In [ ]:
def remove_altitude(data):
    new_data = []
    for item in data:
        new_polygon = []
        for coordinates in item['polygon']:
            new_coordinates = coordinates[:2]  # Remove altitude value
            new_polygon.append(new_coordinates)
        item['polygon'] = new_polygon
        old_home_plate = item['home_plate']
        item['home_plate'] = item['home_plate'][:2]  # Remove altitude value from home_plate
        print(f"Old home_plate: {old_home_plate}, New home_plate: {item['home_plate']}")  # Debugging line
        new_data.append(item)
    return new_data

updated_data = remove_altitude(data)
data_with_distance = add_distances_to_data(updated_data)

print(data_with_distance)


In [ ]:
from shapely.geometry import Point
import geopy.distance

def add_distances_to_data(data):
    def get_distance(point1, point2):
        return geopy.distance.distance(point1, point2).km

    def process_item(item):
        home_lat = item['home_plate'][1]
        home_lon = item['home_plate'][0]
        home_point = Point(home_lon if not isinstance(home_lon, list) else home_lon[0], 
                           home_lat if not isinstance(home_lat, list) else home_lat[0])
        distances = []
        for polygon in item['polygon']:
            if len(polygon) > 0:
                polygon_lat, polygon_lon = polygon[0][1], polygon[0][0]
                polygon_point = Point(polygon_lon if not isinstance(polygon_lon, list) else polygon_lon[0], 
                                      polygon_lat if not isinstance(polygon_lat, list) else polygon_lat[0])
                distance = get_distance(home_point, polygon_point)
                distances.append(distance)
        return {'home_plate': item['home_plate'], 'polygon': item['polygon'], 'distances': distances}
        
    return [process_item(item) for item in data]


In [ ]:
print(home_plate_to_lat_lon("84N8.001'"))

In [ ]:
# import json

# # Load data from JSON file
# with open('output.json', 'r') as file:
#     data = json.load(file)



def remove_altitude(data):
    new_data = []
    for item in data:
        new_polygon = []
        for coordinates in item['polygon']:
            new_coordinates = coordinates[:2]  # Remove altitude value
            new_polygon.append(new_coordinates)
        item['polygon'] = new_polygon
        item['home_plate'] = item['home_plate'][:2]  # Remove altitude value from home_plate
        new_data.append(item)
    return new_data

updated_data = remove_altitude(data)
# print(updated_data)


In [ ]:
import json

# Load data from JSON file
with open('output.json', 'r') as file:
    data = json.load(file)

# Call remove_altitude function
updated_data = remove_altitude(data)

# Print the updated data

# Save the updated data to a text file

with open('output.json', 'w') as outfile:
    json.dump(updated_data, outfile)


print(updated_data)

In [ ]:
import json
from shapely.geometry import Point
from shapely.geometry import LineString



data_with_distances = []

def add_distances_to_data(data):
    for item in data:
        home_lat, home_lon = item['home_plate']
        home_point = Point(home_lat, home_lon)
        distances = []
        for point in item['polygon']:
            lat, lon = point
            point = Point(lat, lon)
            distance = home_point.distance(point) * 111139  # Convert to meters
            distance_ft = round(distance * 3.28084)


# Call add_distances_to_data function
data_with_distances = add_distances_to_data(updated_data)

# Print the data with distances
# print(data_with_distances)






In [ ]:
### NOT WORKING - ABANDONED

import math

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth's radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

def meters_to_feet(meters):
    return meters * 3.28084

def add_distances_to_data(data):
    for item in data:
        home_lat, home_lon = item['home_plate'][:2]
        distances = []
        for point in item['polygon']:
            lat, lon = point
            distance_meters = haversine_distance(home_lat, home_lon, lat, lon)
            distance_feet = meters_to_feet(distance_meters)
            distances.append(distance_feet)
        item['distances'] = distances
    return data

# Call add_distances_to_data function
data_with_distances = add_distances_to_data(updated_data)

# Print the data with distances
print(data_with_distances)


In [ ]:
### NOT WORKING WEDNESDAY 4-12 4:30 pm

import json
import csv
from urllib.request import urlopen
import geopy.distance

url = "https://raw.githubusercontent.com/JSmith1826/BB_parks/main/data/360_test/output.json"

# Read the GeoJSON file
data = json.load(urlopen(url))

# Calculate distances function
def calculate_distances(feature):
    home_plate_point = (feature['home_plate_lat'], feature['home_plate_lon'])
    polygon = feature['geometry']['coordinates'][0]
    distances = []
    
    for vertex in polygon:
        if isinstance(vertex, (list, tuple)) and len(vertex) == 2:
            try:
                lat, lon = float(vertex[1]), float(vertex[0])
            except ValueError:
                continue
        else:
            continue

        vertex_point = (lat, lon)
        distance = geopy.distance.distance(home_plate_point, vertex_point).feet
        distances.append(distance)

    return distances


# Calculate distances and add them to the data
for feature in data:
    distances = calculate_distances(feature)
    feature['distances'] = [round(distance, 2) for distance in distances]

# Output the data to a CSV file
with open('distances.csv', 'w', newline='') as csvfile:
    fieldnames = ['field_name', 'fair_foul', 'home_plate', 'polygon', 'distances']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for feature in data:
        writer.writerow(feature)



In [ ]:
import json
from urllib.request import urlopen

url = "https://raw.githubusercontent.com/JSmith1826/BB_parks/main/data/360_test/output.json"

# Read the GeoJSON file
data = json.load(urlopen(url))

# Print the data for inspection
print(data)


In [ ]:
import csv
import json
import geopy.distance
from urllib.request import urlopen

url = "https://raw.githubusercontent.com/JSmith1826/BB_parks/main/data/360_test/output.json"

# Read the GeoJSON file
data = json.load(urlopen(url))
print(data)  # Print the data to understand the structure

In [ ]:
import csv
import json
import geopy.distance
from urllib.request import urlopen

url = "https://raw.githubusercontent.com/JSmith1826/BB_parks/main/data/360_test/output.json"

# Read the GeoJSON file
data = json.load(urlopen(url))


for feature in data:
    field_name = feature['properties']['layer']
    fair_foul = feature['properties']['Name']
    home_plate = feature['geometry']['coordinates'][0][0][0:2]  # Only take x and y coordinates
    polygon = [point[0:2] for point in feature['geometry']['coordinates'][0]]  # Only take x and y coordinates for each point
    data_list.append({'field_name': field_name, 'fair_foul': fair_foul, 'home_plate': home_plate, 'polygon': polygon})


def calculate_distances(feature):
    home_plate = feature['home_plate']
    home_plate_point = (home_plate[1], home_plate[0])

    polygon = feature['polygon']
    distances = []

    for vertex in polygon:
        vertex_point = (vertex[1], vertex[0])  # Extract only the x and y coordinates
        distance = geopy.distance.distance(home_plate_point, vertex_point).feet
        distances.append(distance)

    return distances




# Calculate distances and add them to the data
for feature in data_list:
    distances = calculate_distances(feature)
    feature['distances'] = [round(distance, 2) for distance in distances]

# Output the data to a CSV file
csv_columns = ['field_name', 'fair_foul', 'home_plate', 'polygon', 'distances']
csv_file = "output.csv"

with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for data in data_list:
        writer.writerow(data)


In [ ]:
import json

with open('output.json') as f:
    data = json.load(f)

polygons_by_layer = {}

for feature in data:
    layer = feature['properties']['layer']
    polygon = feature['geometry']['coordinates'][0]
    if layer not in polygons_by_layer:
        polygons_by_layer[layer] = polygon
    else:
        polygons_by_layer[layer].extend(polygon)



In [ ]:
import pandas as pd

# Load the csv file
df = pd.read_csv('baseball_fields.csv')

# Clean up the layer name
df['clean_name'] = df['layers'].str.replace('ballparks - ', '').str.strip()

# Save the updated dataframe to a new csv file
df.to_csv('updated_baseball_fields.csv', index=False)

